# Deploying Application

To deploy the ADCIRC interactive application the following must be done:

1. Zip the application contents into a folder including
    - tapisjob_app.sh - Main entrypoint script that launches the container.
2. Uplaod this zipped folder to DesignSafe using tapis
    - If testing, upload to a test directory in my own user space: `tapis://designsafe.storage.default/clos21/tapisv3/apps/adcirc-interactive-test.zip`
    - If production, upload to a directory other users can access: `tapis://cloud.data/corral/tacc/aci/CEP/applications/v3/adcirc-interactive/v1/adcirc-interactive.zip`
3. Deploy the application using Tapis 




## Log into DesignSafe using Tapis

In [1]:
import os
from dotenv import load_dotenv
from tapipy.tapis import Tapis


# Load the .env file
load_dotenv()

# Now you can access the variables using os.getenv()
tacc_psw = os.getenv('TACC_PSW')

t = Tapis(base_url='https://designsafe.tapis.io', username='clos21', password=tacc_psw)
t.get_tokens()

## Stage application data localy

Note that it is best practice to make a test version of the application deployed to a path in your MyData, and make sure that works before deploying a production app that is at a location everyone can access.

In [50]:
deploy_type = 'test'
# deploy_type = 'prod'

if deploy_type == 'test':
    app_id = 'adcirc-interactive-test'
    deploy_path = '/clos21/tapisv3/apps/adcirc-interactive-test.zip'
    deploy_uri = f'tapis://designsafe.storage.default{deploy_path}'
    deploy_system = 'designsafe.storage.default'
    label = '(TEST) Interactive VM for ADCIRC - v56'
else:
    app_id = 'adcirc-interactive'
    deploy_path = '/corral/tacc/aci/CEP/applications/v3/adcirc-interactive/v1/adcirc-interactive.zip'
    deploy_uri = f'tapis://cloud.data/{deploy_path}'
    deploy_system = 'cloud.data'
    label = 'Interactive VM for ADCIRC - v56'

In [41]:
# Copy tapisjob_app.sh into a folder with name equal to app_id, and zip this folder in current directory:
os.system(f'rm -rf {app_id}; rm {app_id}.zip')
os.system(f'mkdir -p {app_id}; cp tapisjob_app.sh {app_id}/; chmod +x {app_id}/tapisjob_app.sh')
os.system(f'cd {app_id} && zip -r ../{app_id}.zip . && cd ..')

  adding: tapisjob_app.sh (deflated 66%)


rm: adcirc-interactive.zip: No such file or directory


0

## Send Application Data to DesignSafe 

In [45]:
# Upload zip file using Tapis
t.upload(source_file_path=f"{app_id}.zip", system_id=deploy_system, dest_file_path=deploy_path)

ForbiddenError: message: FILES_CLIENT_SSH_PERM_DENIED OboTenant: designsafe OboUser: clos21 Operation: mkdir System: cloud.data EffectiveUser: clos21 Host: cloud.data.tacc.utexas.edu Path: /corral-tacc Error: SFTP error (SSH_FX_PERMISSION_DENIED): Permission denied

## Build Application Configuration

In [51]:
app_config = {
    "id": app_id,
    "version": "56.00",
    "description": "ADCIRC Interactive provides users with a JupyterLab environment running on a small VM with pre-installed adcirc/padcirc for testing before submitting HPC jobs.",
    "owner": "${apiUserId}",
    "enabled": True,
    "runtime": "ZIP",
    "runtimeVersion": None,
    "runtimeOptions": None,
    "containerImage": deploy_uri,
    "jobType": "FORK",
    "maxJobs": -1,
    "maxJobsPerUser": -1,
    "strictFileInputs": True,
    "jobAttributes": {
        "execSystemConstraints": None,
        "execSystemId": "wma-exec-01",
        "execSystemExecDir": "${JobWorkingDir}",
        "execSystemInputDir": "${JobWorkingDir}",
        "execSystemOutputDir": "${JobWorkingDir}",
        "execSystemLogicalQueue": None,
        "archiveSystemId": "cloud.data",
        "archiveSystemDir": "HOST_EVAL($HOME)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}",
        "archiveOnAppError": True,
        "isMpi": False,
        "mpiCmd": None,
        "cmdPrefix": None,
        "parameterSet": {
            "appArgs": [],
            "containerArgs": [],
            "schedulerOptions": [],
            "envVariables": [
                {
                    "key": "_UserProjects",
                    "value": "",
                    "description": "User Project UUIDs and Project IDs",
                    "inputMode": "INCLUDE_ON_DEMAND",
                    "notes": {
                        "isHidden": True
                    }
                },
                {
                    "key": "_DOCKER_IMAGE",
                    "value": "docker://clos21/ds-adcirc-interactive:latest",
                    "description": "Docker image to use for the job",
                    "inputMode": "FIXED",
                    "notes": {
                        "isHidden": True
                    }
                }
            ],
            "archiveFilter": {
                "includes": [],
                "excludes": [
                    "adcirc_interactive.zip",
                    "tapisjob.env"
                ],
                "includeLaunchFiles": True
            }
        },
        "fileInputs": [],
        "fileInputArrays": [],
        "maxMinutes": 2880,
        "subscriptions": [],
        "tags": []
    },
    "tags": [
        "portalName: DesignSafe",
        "portalName: CEP"
    ],
    "notes": {
        "label": label,
        "helpUrl": "https://www.designsafe-ci.org/user-guide/tools/simulation/#adcirc-through-the-interactive-vm",
        "hideNodeCountAndCoresPerNode": True,
        "hideMaxMinutes": True,
        "isInteractive": True,
        "icon": "ADCIRC",
        "category": "Simulation"
    }
}
app_config

{'id': 'adcirc-interactive',
 'version': '56.00',
 'description': 'ADCIRC Interactive provides users with a JupyterLab environment running on a small VM with pre-installed adcirc/padcirc for testing before submitting HPC jobs.',
 'owner': '${apiUserId}',
 'enabled': True,
 'runtime': 'ZIP',
 'runtimeVersion': None,
 'runtimeOptions': None,
 'containerImage': 'tapis://cloud.data//corral/tacc/aci/CEP/applications/v3/adcirc-interactive/v1/adcirc-interactive.zip',
 'jobType': 'FORK',
 'maxJobs': -1,
 'maxJobsPerUser': -1,
 'strictFileInputs': True,
 'jobAttributes': {'execSystemConstraints': None,
  'execSystemId': 'wma-exec-01',
  'execSystemExecDir': '${JobWorkingDir}',
  'execSystemInputDir': '${JobWorkingDir}',
  'execSystemOutputDir': '${JobWorkingDir}',
  'execSystemLogicalQueue': None,
  'archiveSystemId': 'cloud.data',
  'archiveSystemDir': 'HOST_EVAL($HOME)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}',
  'archiveOnAppError': True,
  'isMpi': False,
  'mpiCmd': None,


In [52]:
try:
    t.apps.getAppLatestVersion(appId=app_config['id'])
except Exception:
    t.apps.createAppVersion(**app_config)
else:
    t.apps.putApp(appId=app_config['id'], appVersion='56.00', **app_config)

## Verify the deployed application configuration

In [48]:
t.apps.getAppLatestVersion(appId=app_config['id'])


containerImage: tapis://cloud.data/corral-tacc/tacc/aci/CEP/applications/v3/adcirc-interactive/v1/adcirc-interactive.zip
created: 2024-10-03T15:26:54.307943Z
deleted: False
description: ADCIRC Interactive provides users with a JupyterLab environment running on a small VM with pre-installed adcirc/padcirc for testing before submitting HPC jobs.
enabled: True
id: adcirc-interactive
isPublic: True
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($HOME)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: cloud.data
cmdPrefix: None
coresPerNode: 1
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: wma-exec-01
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: None
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: []
isMpi: False
maxMinutes: 2880
memoryMB: 100
mpiCmd: None
nodeCount: